# Greetings
In this task, your objective is to employ a **Classifier** to forecast the output. Please ensure to write/edit only between the
```# START CODE HERE``` & ```# END CODE HERE``` markers.<br>
Modifying the test cells will result in failure of the assignment.

Before running the first cell, make sure to upload the ```utils.zip``` file.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**GOOD LUCK**


**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [2]:
!unzip -j /utils.zip -d .

unzip:  cannot find or open /utils.zip, /utils.zip.zip or /utils.zip.ZIP.


In [5]:
#necessary libraries to be uploaded
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# import essential libraries
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler


In [1]:
# Data Loading and Preprocessing

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [3]:
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
2834,24.0,34,10.5,30.0,partly cloudy,967.99,7,Spring,13.0,coastal,Snowy
7528,26.0,76,11.5,30.0,partly cloudy,1000.71,4,Autumn,7.5,mountain,Cloudy
7478,26.0,89,8.0,83.0,partly cloudy,1010.52,8,Spring,11.5,mountain,Sunny
4507,29.0,28,2.5,3.0,partly cloudy,1029.98,7,Autumn,6.5,mountain,Sunny
1258,25.0,28,0.0,0.0,clear,1018.63,7,Spring,7.5,mountain,Sunny


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [8]:
# START CODE HERE
non_object_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
object_columns = data.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

scaler = StandardScaler()
data[non_object_columns] = scaler.fit_transform(data[non_object_columns])

# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [9]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [10]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE
column_transformer = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(), object_columns),
        ('scaler', StandardScaler(), non_object_columns)
    ],
    remainder='passthrough'
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [11]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [12]:
X = data_scaled_labeled.drop(columns=['Weather Type'])
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=10)

**TEST** (DON'T EDIT THIS CELL)

In [13]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [14]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,2.0,1.0,1.0,0.855442,-0.332327,0.530916,0.198953,0.208400,-0.520104,-0.582231
12278,2.0,1.0,0.0,1.833259,1.995157,1.037543,1.388485,-0.317163,1.035729,0.010999
2859,0.0,0.0,0.0,0.740404,0.707613,0.892792,1.169360,0.192539,-0.001493,0.752538
5495,0.0,1.0,0.0,1.143035,-1.966517,-1.350840,-1.178401,0.314319,0.257813,0.159307
8469,0.0,0.0,0.0,0.797923,-0.530410,-1.350840,-1.460132,0.137698,0.517118,0.159307


In [17]:
y_train.unique()

array([1., 3., 2., 0.])

# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [20]:
# START CODE HERE

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=10)

model.fit(X_train, y_train)

score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 92.42424242424242


**TEST** (DON'T EDIT THIS CELL)

In [21]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [22]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,2.0,3.0,0.0,1.545665,-2.362685,-0.627087,0.105042,3.580608,-1.038715,0.604230
2331,2.0,0.0,0.0,0.797923,-0.282806,0.024290,-0.458420,-0.058280,-0.001493,0.752538
217,3.0,2.0,0.0,0.337774,-2.016038,-0.192836,-1.334918,0.550621,1.813646,1.345768
1760,0.0,3.0,2.0,0.855442,-1.669392,-0.192836,-1.679256,0.233132,0.776424,1.049153
3963,2.0,0.0,2.0,-0.294931,-0.332327,-0.554712,1.325878,0.019412,-0.260799,-0.433923


In [23]:
y = model.predict(input)
y

array([3., 0., 3., 3., 1.])

Here, the outputs are in numbers. We can perform inverse column

---

transfer for getiing the string values.
